<a href="https://colab.research.google.com/github/roberthsu2003/huggingFace_api/blob/main/%E9%80%9A%E7%94%A8%E8%AA%9E%E8%A8%80/llama/demo2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 可以使用CPU執行

In [ ]:
%%capture
%pip install -U transformers accelerate

In [ ]:
from huggingface_hub import login
from google.colab import userdata

# 使用環境變數中的token進行登入
login(userdata.get('huggingface_token'))

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, TextStreamer
import torch


base_model = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    return_dict=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)


In [ ]:
# Set pad_token_id if not already set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
if model.config.pad_token_id is None:
    model.config.pad_token_id = model.config.eos_token_id

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
messages = [{"role": "user", "content": "請介紹台灣的101大樓?"}]

prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

outputs = pipe(prompt, max_new_tokens=120, do_sample=True)

print(outputs[0]["generated_text"])


In [ ]:
from IPython.display import Markdown, display

messages = [
    {
        "role": "system",
        "content": "您是一位熟練的 Python 開發人員，專門從事資料庫管理和最佳化。",
    },
    {
        "role": "user",
        "content": "我的資料庫遇到了排序問題。您能提供 Python 程式碼來幫助解決這個問題嗎？",
    },
]

prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

outputs = pipe(prompt, max_new_tokens=512, do_sample=True)

display(
    Markdown(
            outputs[0]["generated_text"].split(
                "<|start_header_id|>assistant<|end_header_id|>"
            )[1]
        )
    )
